# Building a benchmarking pipeline

In [14]:
import time
import pandas as pd

from sklearn.preprocessing import StandardScaler

from pipeline.pipeline import Pipeline
from pipeline import synthetic_data
from pipeline.models.models import SUPPORTED_MODELS
from pipeline.models.params import SUPPORTED_PARAMS, PARAMS
from pipeline.helpers.misc import build_synth_data_name
from pipeline.models import params_generators
from pipeline.synthetic_data.base_generator import generate

from neuralprophet.utils import set_log_level

In [2]:
set_log_level("ERROR")

## Experiment definition

### Synthetic data
Available datasets:

In [3]:
synthetic_data.__all__

['gen_one_shape_ar',
 'gen_one_shape_ar_outlier_0p1',
 'gen_one_shape_ar_outlier_1p',
 'gen_cancel_shape_ar',
 'gen_cancel_shape_ar_outlier_0p1',
 'gen_cancel_shape_ar_outlier_1p',
 'gen_one_shape_ar_trend',
 'gen_one_shape_ar_trend_cp',
 'generate_intermittent',
 'gen_one_shape_heteroscedacity',
 'gen_one_shape_heteroscedacity_op',
 'gen_struc_break_mean',
 'gen_struc_break_var']

Choose a synthetic data generator for the experiment

In [4]:
data_func = "gen_one_shape_ar"

### Models
Available models:

In [5]:
SUPPORTED_MODELS

['NeuralProphetModel',
 'TorchProphetModel',
 'LightGBMModel',
 'RNNModel',
 'TransformerModel',
 'NaiveModel',
 'SeasonalNaiveModel']

In [6]:
model_name = "NeuralProphetModel"

### Model params
For every model there are available default params sets:

In [7]:
PARAMS

{'NeuralProphetModel': {'NP': {'n_forecasts': 1,
   'epochs': 30,
   'global_normalization': True,
   'normalize': 'off',
   'n_lags': 4},
  'NP_localST': {'n_forecasts': 1,
   'epochs': 30,
   'global_normalization': True,
   'normalize': 'off',
   'trend_global_local': 'local',
   'season_global_local': 'local',
   'n_lags': 4},
  'NP_FNN': {'n_forecasts': 1,
   'epochs': 30,
   'global_normalization': True,
   'normalize': 'off',
   'yearly_seasonality': False,
   'weekly_seasonality': False,
   'daily_seasonality': False,
   'n_changepoints': 0,
   'growth': 'off',
   'n_lags': 4},
  'NP_FNN_wb': {'n_forecasts': 1,
   'epochs': 30,
   'global_normalization': True,
   'normalize': 'off',
   'yearly_seasonality': False,
   'weekly_seasonality': False,
   'daily_seasonality': False,
   'n_changepoints': 0,
   'growth': 'off',
   'n_lags': 24},
  'NP_FNN_sw_wb': {'n_forecasts': 1,
   'epochs': 30,
   'global_normalization': True,
   'normalize': 'off',
   'yearly_seasonality': False,
 

Choose a params set available for chosen model

In [8]:
model_base_params = "NP"

### Experiment params generator functions
Experiement params generator allow to create experiment combinations

In [9]:
params_generators.__all__

['gen_model_and_params_default',
 'gen_model_and_params_pytorch_batch_norm',
 'gen_model_and_params_norm',
 'gen_model_and_params_none',
 'gen_model_and_params_scalers',
 'gen_model_and_params_scalers_reweighting']

* `gen_model_and_params_default` -> default combination valid for any model. Does not add any addtional combinations unless specified by the user in `kwargs`. Allows to add combination of scalers by setting argument `with_scalers` to True
* `gen_model_and_params_pytorch_batch_norm` -> adds an additional experiments with pytorch batch layer (with affine layer and without it) to the benchmark. Allowed only for `NeuralProphet` model
* `gen_model_and_params_norm` -> adds combinations of all available normalization layers in `NuralProphet` model (reversible batch and instance normalization, pytorch batch normalizarion, all with or without affine layer)
* `gen_model_and_params_none` -> only experiment in the benchmar is a model with its parameter. Does not add any additional experiments. Ignores `with_scalers` and `kwargs` if set by the user
* `gen_model_and_params_scalers` -> generats all combinations of scalers (scaling_levels: [`per_time_series`, `per_dataset`], scalers: [`StandardScaler`, `MinMaxScaler`, ...]
* `gen_model_and_params_scalers_reweighting` -> similar to before but with combinations of loss re-weighting for `NeuralProphet` model (weights: [`avg`, `std`, `std * avg`])

Choose an experiment params generator function

In [10]:
params_gen_function = "gen_model_and_params_norm"

## Setup a pipeline

Generate synthetic dataset

In [35]:
data_n_ts_groups = [5, 5]
data_offset_per_group = [0, 0]
data_amplitude_per_group = [10, 10]
data_trend_gradient_per_group = None
proportion_break = None

freq = "H"
series_length = 24 * 7 * 15
series_start = pd.to_datetime("2011-01-01 01:00:00")

In [36]:
data = generate(
    series_start=series_start,
    series_length=series_length,
    data_trend_gradient_per_group=data_trend_gradient_per_group,
    data_func=data_func,
    n_ts_groups=data_n_ts_groups,
    offset_per_group=data_offset_per_group,
    amplitude_per_group=data_amplitude_per_group,
    proportion_break=proportion_break,
    freq=freq,
)

Generate pipeline name which also serves as results directory name

In [37]:
pipeline_name = build_synth_data_name(
    data_func,
    model_base_params,
    data_n_ts_groups,
    data_amplitude_per_group,
    data_offset_per_group,
    data_trend_gradient_per_group,
    proportion_break,
)

In [38]:
pipeline = Pipeline(
    model_name=model_name,
    params_name=model_base_params,
    data=data,
    freq=freq,
    pipeline_name=pipeline_name,
    base_dir_name=".",
)

Define `kwargs` if your experiments require additional params

In [39]:
kwargs = {"scalers": [StandardScaler(), None], "scaling_levels": ["per_time_series"]}

### Execute the pipeline

In [40]:
start_time = time.time()
pipeline.run(
    save=True, test_percentage=0.25, params_generator_name=params_gen_function, with_scalers=False, **kwargs
)
end_time = time.time()

INFO - (tot.benchmark.run) - Experiment list:
INFO - (tot.benchmark.run) - exp 1/13: NeuralProphetModel_None
INFO - (tot.benchmark.run) - exp 2/13: NeuralProphetModel_revin_batch_affine_StandardScaler_per_time_series
INFO - (tot.benchmark.run) - exp 3/13: NeuralProphetModel_revin_batch_affine_None
INFO - (tot.benchmark.run) - exp 4/13: NeuralProphetModel_revin_batch_StandardScaler_per_time_series
INFO - (tot.benchmark.run) - exp 5/13: NeuralProphetModel_revin_batch_None
INFO - (tot.benchmark.run) - exp 6/13: NeuralProphetModel_pytorch_batch_affine_StandardScaler_per_time_series
INFO - (tot.benchmark.run) - exp 7/13: NeuralProphetModel_pytorch_batch_affine_None
INFO - (tot.benchmark.run) - exp 8/13: NeuralProphetModel_pytorch_batch_StandardScaler_per_time_series
INFO - (tot.benchmark.run) - exp 9/13: NeuralProphetModel_pytorch_batch_None
INFO - (tot.benchmark.run) - exp 10/13: NeuralProphetModel_revin_instance_affine_StandardScaler_per_time_series
INFO - (tot.benchmark.run) - exp 11/13:

INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequenc

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of 

INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major freq

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.843% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.843% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.843% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.843% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.843% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.843%

...

INFO - (tot.benchmark._run_exp) - --------------------------------------------------------
INFO - (tot.benchmark._run_exp) - finished exp 11: NeuralProphetModel_revin_instance_affine_None
INFO - (tot.benchmark._run_exp) - test results 11: {'data': 'synthetic', 'model': 'NeuralProphetModel', 'norm_mode': 'revin', 'norm_type': 'instance', 'norm_affine': True, 'scaler': 'no scaler', 'scaling level': 'none', 'weighted': 'none', 'MAE': 1.487355351448059, 'RMSE': 1.8269747377997867, 'MASE': 2.2826766075176015}
INFO - (tot.benchmark._run_exp) - --------------------------------------------------------
INFO - (tot.benchmark._run_exp) - --------------------------------------------------------
INFO - (tot.benchmark._run_exp) - finished exp 12: NeuralProphetModel_revin_instance_StandardScaler_per_time_series
INFO - (tot.benchmark._run_exp) - test results 12: {'data': 'synthetic', 'model': 'NeuralProphetModel', 'norm_mode': 'revin', 'norm_type': 'instance', 'norm_affine': False, 'scaler': StandardS

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major 

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.842% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.842% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.842% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.842% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.842% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.842%

INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.843% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.843% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.843% of the data.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.842% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.842% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major freq

revin batch True
None None None
revin instance True
revin batch False
revin instance False
revin batch True
revin batch False
revin instance False
pytorch batch False
revin instance True
pytorch batch False
pytorch batch True
pytorch batch True


In [41]:
print("Pipeline execution time: ", end_time - start_time)

Pipeline execution time:  753.9002010822296


In [42]:
pipeline.summary()

Pipeline gen_one_shape_ar_NP_n_ts_[5, 5]_am_[10, 10]_of_[0, 0]_gr_None_None.
Model: NeuralProphetModel with kwargs: {'scalers': [StandardScaler(), None], 'scaling_levels': ['per_time_series']}
Dataset: synthetic
Results train:
         data               model norm_mode norm_type norm_affine  \
0   synthetic  NeuralProphetModel      none      none        none   
1   synthetic  NeuralProphetModel     revin     batch        True   
2   synthetic  NeuralProphetModel     revin     batch        True   
3   synthetic  NeuralProphetModel     revin     batch       False   
4   synthetic  NeuralProphetModel     revin     batch       False   
5   synthetic  NeuralProphetModel   pytorch     batch        True   
6   synthetic  NeuralProphetModel   pytorch     batch        True   
7   synthetic  NeuralProphetModel   pytorch     batch       False   
8   synthetic  NeuralProphetModel   pytorch     batch       False   
9   synthetic  NeuralProphetModel     revin  instance        True   
10  synthetic 

Display benchmarking pipeline results

In [43]:
pipeline.results_test

,data,model,norm_mode,norm_type,norm_affine,scaler,scaling level,weighted,MAE,RMSE,MASE
0,synthetic,NeuralProphetModel,none,none,none,no scaler,none,none,0.261232,0.326910,0.400919
1,synthetic,NeuralProphetModel,revin,batch,True,StandardScaler(),per_time_series,None,0.259867,0.325626,0.398823
2,synthetic,NeuralProphetModel,revin,batch,True,no scaler,none,none,0.259768,0.325521,0.398672
3,synthetic,NeuralProphetModel,revin,batch,False,StandardScaler(),per_time_series,None,0.259864,0.325628,0.398818
4,synthetic,NeuralProphetModel,revin,batch,False,no scaler,none,none,0.259765,0.325523,0.398667
5,synthetic,NeuralProphetModel,pytorch,batch,True,StandardScaler(),per_time_series,None,0.262528,0.328286,0.402908
6,synthetic,NeuralProphetModel,pytorch,batch,True,no scaler,none,none,0.262331,0.328059,0.402606
7,synthetic,NeuralProphetModel,pytorch,batch,False,StandardScaler(),per_time_series,None,0.262483,0.328228,0.402839
8,synthetic,NeuralProphetModel,pytorch,batch,False,no scaler,none,none,0.262289,0.327997,0.402540
9,synthetic,NeuralProphetModel,revin,instance,True,StandardScaler(),per_time_series,None,1.487544,1.827221,2.282967


### Pipeline re-run with different experiments parameters
A defined pipeline can be re-run with different experiments pipeline generator functions but the same dataset and model

In [ ]:
kwargs = {}
pipeline.run(
    save=True, 
    test_percentage=0.25, 
    metrics=["MAE", "RMSE"], 
    params_generator_name="gen_model_and_params_scalers", 
    with_scalers=False, 
    **kwargs
)

[Errno 17] File exists: './results/gen_one_shape_ar_NP_n_ts_[5, 5]_am_[10, 10]_of_[0, 0]_gr_None_None_'


INFO - (tot.benchmark.run) - Experiment list:
INFO - (tot.benchmark.run) - exp 1/13: NeuralProphetModel_None
INFO - (tot.benchmark.run) - exp 2/13: NeuralProphetModel_StandardScaler_per_time_series
INFO - (tot.benchmark.run) - exp 3/13: NeuralProphetModel_StandardScaler_per_dataset
INFO - (tot.benchmark.run) - exp 4/13: NeuralProphetModel_MinMaxScaler_per_time_series
INFO - (tot.benchmark.run) - exp 5/13: NeuralProphetModel_MinMaxScaler_per_dataset
INFO - (tot.benchmark.run) - exp 6/13: NeuralProphetModel_RobustScalerquantile_range=5 95_per_time_series
INFO - (tot.benchmark.run) - exp 7/13: NeuralProphetModel_RobustScalerquantile_range=5 95_per_dataset
INFO - (tot.benchmark.run) - exp 8/13: NeuralProphetModel_PowerTransformer_per_time_series
INFO - (tot.benchmark.run) - exp 9/13: NeuralProphetModel_PowerTransformer_per_dataset
INFO - (tot.benchmark.run) - exp 10/13: NeuralProphetModel_QuantileTransformeroutput_distribution='normal'_per_time_series
INFO - (tot.benchmark.run) - exp 11/13

In [ ]:
pipeline.results_test

In [ ]:
pipeline.run(
    save=True, 
    test_percentage=0.25, 
    metrics=["MAE", "RMSE"], 
    plot=True,
    params_generator_name="gen_model_and_params_none", 
    with_scalers=False, 
    **kwargs
)

In [ ]:
pipeline.results_test